In [1]:
import pandas as pd # data frames
import numpy as np

In [2]:
# read data
df = pd.read_csv("https://raw.githubusercontent.com/askoshiyama/mli-cohort3/master/boston.csv")
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,T1
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Repeated K Fold CV (5 outer, 5 inner, repeat 5 times):
RepeatedKFold repeats K-Fold n times. It can be used when one requires to run KFold n times, producing different splits in each repetition.
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [3]:
# defining vars
input_vars = ["V1", "V2", "V3", "V4", "V5", "V6", "V7",
              "V8", "V9", "V10", "V11", "V12", "V13"]
target = ["T1"]

In [46]:
X = df.drop(labels=target, axis=1)
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [45]:
y = df.drop(labels=input_vars, axis=1)
y.head()

,T1
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [84]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from interpret.glassbox import ExplainableBoostingRegressor 
from xgboost import XGBRegressor
from sklearn.svm import SVR

# define the models for selection along with their hyperparameters
# we try to find meaningful ranges for each hyperparameter. Among the models chosen for selection,
# we also include an MLP neural network as it presents the most recent advancement in supervised learning domain.
models_and_parameters = {
    'dummy': (DummyRegressor(), {}),
    'linReg': (LinearRegression(), {}),
    'ridge': (Ridge(), 
              {'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'svr': (SVR(kernel='rbf'),
              {'C': [0.01, 0.05, 0.1, 0.5, 1], 
               'gamma': [0.001, 0.0001, 0.00001]}),
    'krr': (KernelRidge(), 
              {'kernel': ["poly","rbf"],
               'degree': [2,3,4],
               'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'dtr': (DecisionTreeRegressor(),
              {"min_samples_split": [2, 10, 20, 40], 
               "max_depth": [2, 6, 8],
               "min_samples_leaf": [1, 20, 40, 100],
               "max_leaf_nodes": [None, 5, 20, 100]}),
    'rf': (RandomForestRegressor(n_estimators=100),
              {'max_depth': [5, 10, 50, 100, 200, 500]}),
    'gbr': (GradientBoostingRegressor(),
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
    'ebr': (ExplainableBoostingRegressor(), {}),
    'xgbr': (XGBRegressor(), 
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
}
#     'mlp': (MLPRegressor(early_stopping=True),
#                {'mlp__hidden_layer_sizes': [(2,),(4,),(8,),(16,),(32,),(64,),(128,)],
#                 'mlp__activation': ['relu'],
#                 'mlp__solver':['lbfgs', 'adam', 'sgd'], 
#                 'mlp__alpha':[0.0001],
#                 'mlp__batch_size':['auto'], 
#                 'mlp__learning_rate':['constant'],
#                 'mlp__learning_rate_init':[0.01, 0.001, 0.0001], 
#                 'mlp__max_iter':[800]})

In [82]:
from sklearn import metrics

# performance metrics
perf_metrics = {"MAE": metrics.mean_absolute_error, 
                "EVS": metrics.explained_variance_score, 
                "MSE": metrics.mean_squared_error,
                "MDAE": metrics.median_absolute_error, 
                "R2": metrics.r2_score
               }
metric_greater_is_better = \
               {"MAE": False, 
                "EVS": True, 
                "MSE": False,
                "MDAE": False, 
                "R2": True
               }
scorer_metrics = {}
for pf in perf_metrics:
    scorer_metrics[pf] = metrics.make_scorer(score_func=perf_metrics[pf], greater_is_better=metric_greater_is_better[pf])

In [85]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, GridSearchCV
import numpy as np
from sklearn import metrics
from time import time


#######################################
# configuration
REPEAT = 1
SEED = None  # will ensure different results on each iteration
K_FOLD_NUM = 5
REFIT_METRIC = 'MSE'
#######################################

# use nested cross-validation for grid search and validation
# creates 5 folds for estimating generalization error
outer_folds = RepeatedKFold(n_repeats=REPEAT, n_splits=K_FOLD_NUM, random_state=SEED)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_folds = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

i, z, innercv_results, inner_best_model = 0, 0, {}, {}
df_outer_results = pd.DataFrame(index=[0], columns=["Model", "Time"] + list(perf_metrics.keys()))

X_arr, y_arr = X.values, y.values

# outer loop
for (out_train, out_test) in outer_folds.split(X_arr, y_arr):
    print(f"Repetition {int(i / K_FOLD_NUM) + 1} Fold {i % K_FOLD_NUM + 1}")
    i += 1  # iteration
    # separation: train, test
    Xtrain, Xtest = X_arr[out_train], X_arr[out_test]
    ytrain, y_test = y_arr[out_train], y_arr[out_test]
    
    # inner loop -- all models
    fold_id = "fold_" + str(i)
    innercv_results[fold_id] = {}
    inner_best_model[fold_id] = {}
    start, end = [], []
    
    for model_name, (model, params) in models_and_parameters.items():
        start.append(time())
        print("\t", model_name)
        innercv_results[fold_id][model_name] = GridSearchCV(estimator=model, 
                                                            param_grid=params, 
                                                            scoring=scorer_metrics, 
                                                            iid=False,
                                                            cv=inner_folds, 
                                                            refit=REFIT_METRIC)    
        innercv_results[fold_id][model_name].fit(Xtrain, ytrain.ravel())
        inner_best_model[fold_id][model_name] = innercv_results[fold_id][model_name].best_estimator_
        end.append(time())        
    
    # outer loop prediction    
    for v, (model_name, (model, _)) in enumerate(models_and_parameters.items()):
        # prediction
        y_test_pred = inner_best_model[fold_id][model_name].predict(Xtest)
        df_outer_results.loc[z, "Model"] = model_name
        df_outer_results.loc[z, "Time"] = end[v] - start[v]
        
        # compute performance metrics
        for pf in perf_metrics.keys():
            df_outer_results.loc[z, pf] = perf_metrics[pf](y_test, y_test_pred)
        z += 1        

# final organisation
df_outer_results[list(perf_metrics.keys()) + ["Time"]] = df_outer_results[list(perf_metrics.keys()) + ["Time"]].astype(float)
print("done.")

Repetition 1 Fold 1
	 dummy
	 linReg
	 ridge
	 svr
	 krr


C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.72852e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_mod

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.72852e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_mod

...

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.15169e-19): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.07878e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=6.3911e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.07749e-17): result may not be accurat

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.15169e-19): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.93765e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.22308e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.01135e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.23654e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\e

	 dtr
	 rf
	 gbr
	 ebr
	 xgbr
[18:00:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:

[18:00:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:19] WARNI

[18:00:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:00:23] WARNI

In [86]:
df_outer_results.pivot_table(index=["Model"], aggfunc="mean")

,EVS,MAE,MDAE,MSE,R2,Time
Model,,,,,,
dtr,7.358596e-01,2.906646,1.918784,22.241523,0.734435,3.436193
dummy,-8.881784e-17,6.656448,4.762839,84.628499,-0.007098,0.012196
ebr,8.260339e-01,2.503000,1.706848,14.565834,0.822692,45.543682
gbr,8.830504e-01,2.137486,1.525193,9.952860,0.881654,14.825601
krr,8.164701e-01,2.610738,1.815452,15.292612,0.814656,41.772621
linReg,7.268719e-01,3.385594,2.596291,23.206449,0.722423,0.018948
rf,8.754108e-01,2.255113,1.590400,10.621005,0.873531,10.822156
ridge,7.267465e-01,3.382341,2.567266,23.234433,0.722080,0.835336
svr,3.156644e-01,4.919969,2.960138,60.699866,0.277519,1.197190
